Create new data frame from analysis insights 

In [245]:
# # Import libraries for pre-processing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Import libraries for model selection

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [246]:
# import csv file

df = pd.read_csv('../Preprocessing/masterupdateafter2000.csv')
df_2 = pd.read_csv('../Preprocessing/playerswithclub.csv')
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,Avg>2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA
0,03/08/07,Mechelen,Anderlecht,0.0,1.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04/08/07,Charleroi,Roeselare,1.0,1.0,D,1.0,0.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04/08/07,Club Brugge,Bergen,2.0,1.0,H,1.0,1.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/08/07,Dender,Germinal,1.0,2.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04/08/07,FC Brussels,Westerlo,0.0,1.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
import pandas as pd

# Full-time result for Away Team
away_team_ftr = df[df['FTR'] == 'A']['AwayTeam'].value_counts()

# Full-time result for Home Team
home_team_ftr = df[df['FTR'] == 'H']['HomeTeam'].value_counts()

# Goals scored by Away Team
goals_away = df.groupby('AwayTeam')['FTAG'].sum().sort_values(ascending=False)

# Goals scored by Home Team
goals_home = df.groupby('HomeTeam')['FTHG'].sum().sort_values(ascending=False)

# Shots on target by Away Team
shots_on_target_away = df.groupby('AwayTeam')['AST'].sum().sort_values(ascending=False)

# Shots on target by Home Team
shots_on_target_home = df.groupby('HomeTeam')['HST'].sum().sort_values(ascending=False)

# Head-to-Head Records between HomeTeam and AwayTeam
h2h_records = df.groupby(['HomeTeam', 'AwayTeam'])['FTR'].value_counts().unstack(fill_value=0)

# calculate win streaks for home team and away team

def win_streaks(df, team):
    wins = df['FTR'][df['HomeTeam'] == team].apply(lambda x: 1 if x == 'H' else 0).values
    streaks = []
    streak = 0
    for i in range(len(wins)):
        if wins[i] == 1:
            streak += 1
        else:
            streaks.append(streak)
            streak = 0
    streaks.append(streak)
    return max(streaks)

# Calculate win streaks for home team

df['HomeTeamWinStreak'] = df['HomeTeam'].apply(lambda x: win_streaks(df, x))

# Calculate win streaks for away team

df['AwayTeamWinStreak'] = df['AwayTeam'].apply(lambda x: win_streaks(df, x))

# Calculate loss streaks

def loss_streaks(df, team):
    wins = df['FTR'][df['HomeTeam'] == team].apply(lambda x: 1 if x == 'A' else 0).values
    streaks = []
    streak = 0
    for i in range(len(wins)):
        if wins[i] == 1:
            streak += 1
        else:
            streaks.append(streak)
            streak = 0
    streaks.append(streak)
    return max(streaks)

# Calculate loss streaks for home team

df['HomeTeamLossStreak'] = df['HomeTeam'].apply(lambda x: loss_streaks(df, x))

# Calculate loss streaks for away team

df['AwayTeamLossStreak'] = df['AwayTeam'].apply(lambda x: loss_streaks(df, x))





In [260]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,HomeTeamWinStreak,AwayTeamWinStreak,HomeTeamLossStreak,AwayTeamLossStreak
0,03/08/07,Mechelen,Anderlecht,0.0,1.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,17,4,3
1,04/08/07,Charleroi,Roeselare,1.0,1.0,D,1.0,0.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9,4,4,4
2,04/08/07,Club Brugge,Bergen,2.0,1.0,H,1.0,1.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,21,7,2,4
3,04/08/07,Dender,Germinal,1.0,2.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,10,3,5
4,04/08/07,FC Brussels,Westerlo,0.0,1.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,5,4,7


In [249]:
df_2.head() 

,name,position,age,club,market,club_value
0,Andreas Skov Olsen,Right Winger,24,Club Brugge KV,18.0,€136.85m
1,Anders Dreyer,Right Winger,26,RSC Anderlecht,15.0,€115.80m
2,Kasper Dolberg,Centre-Forward,26,RSC Anderlecht,12.0,€115.80m
3,Koki Machida,Centre-Back,27,Union Saint-Gilloise,10.0,€77.30m
4,Raphael Onyedika,Defensive Midfield,23,Club Brugge KV,10.0,€136.85m


In [259]:
df_2.head() 

,name,position,age,club,market,club_value
0,Andreas Skov Olsen,Right Winger,24,Club Brugge KV,18.0,136.85
1,Anders Dreyer,Right Winger,26,RSC Anderlecht,15.0,115.80
2,Kasper Dolberg,Centre-Forward,26,RSC Anderlecht,12.0,115.80
3,Koki Machida,Centre-Back,27,Union Saint-Gilloise,10.0,77.30
4,Raphael Onyedika,Defensive Midfield,23,Club Brugge KV,10.0,136.85


In [251]:
# convert club_value to float

df_2['club_value'] = df_2['club_value'].str.replace('€','')
df_2['club_value'] = df_2['club_value'].str.replace('m','')

df_2['club_value'] = df_2['club_value'].astype(float)





In [261]:
# just empty create columns HomeTeamValue and AwayTeamValue 


# fill the columns with the values from the df_2 if club name is the same

for i in range(len(df)):
    for j in range(len(df_2)):
        if df.loc[i, 'HomeTeam'] == df_2.loc[j, 'club']:
            df.loc[i, 'HomeTeamValue'] = df_2.loc[j, 'club_value']
        if df.loc[i, 'AwayTeam'] == df_2.loc[j, 'club']:
            df.loc[i, 'AwayTeamValue'] = df_2.loc[j, 'club_value']



In [262]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,HomeTeamWinStreak,AwayTeamWinStreak,HomeTeamLossStreak,AwayTeamLossStreak
0,03/08/07,Mechelen,Anderlecht,0.0,1.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,17,4,3
1,04/08/07,Charleroi,Roeselare,1.0,1.0,D,1.0,0.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9,4,4,4
2,04/08/07,Club Brugge,Bergen,2.0,1.0,H,1.0,1.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,21,7,2,4
3,04/08/07,Dender,Germinal,1.0,2.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,10,3,5
4,04/08/07,FC Brussels,Westerlo,0.0,1.0,A,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,5,4,7


In [170]:
'''# Full time result for awayteam

plt.figure(figsize=(20, 10))
heatmap_data = pd.crosstab(df['AwayTeam'], df['FTR'])
sns.heatmap(heatmap_data, annot=True, cmap='Blues', fmt="d")
plt.title('Heatmap of Full Time Results by Away Team')
plt.xlabel('Full Time Result')
plt.ylabel('Away Team')
plt.xticks(rotation=90)
plt.show()'''

'# Full time result for awayteam\n\nplt.figure(figsize=(20, 10))\nheatmap_data = pd.crosstab(df[\'AwayTeam\'], df[\'FTR\'])\nsns.heatmap(heatmap_data, annot=True, cmap=\'Blues\', fmt="d")\nplt.title(\'Heatmap of Full Time Results by Away Team\')\nplt.xlabel(\'Full Time Result\')\nplt.ylabel(\'Away Team\')\nplt.xticks(rotation=90)\nplt.show()'

In [171]:
'''# Full time result for hometeam

plt.figure(figsize=(20, 10))
heatmap_data = pd.crosstab(df['HomeTeam'], df['FTR'])
sns.heatmap(heatmap_data, annot=True, cmap='Blues', fmt="d")
plt.title('Heatmap of Full Time Results by Home Team')
plt.xlabel('Full Time Result')
plt.ylabel('Home Team')
plt.xticks(rotation=90)
plt.show()'''

'# Full time result for hometeam\n\nplt.figure(figsize=(20, 10))\nheatmap_data = pd.crosstab(df[\'HomeTeam\'], df[\'FTR\'])\nsns.heatmap(heatmap_data, annot=True, cmap=\'Blues\', fmt="d")\nplt.title(\'Heatmap of Full Time Results by Home Team\')\nplt.xlabel(\'Full Time Result\')\nplt.ylabel(\'Home Team\')\nplt.xticks(rotation=90)\nplt.show()'

In [172]:
'''# Mean of goals scored by awayteam

plt.figure(figsize=(20, 10))
sns.barplot(x=df['FTAG'].groupby(df['AwayTeam']).mean().sort_values(ascending=False).index,y=df['FTAG'].groupby(df['AwayTeam']).mean().sort_values(ascending=False))
plt.title('Mean of Goals Scored by Away Team')
plt.xlabel('Away Team')
plt.ylabel('Goals')
plt.xticks(rotation=90)
plt.show()'''

"# Mean of goals scored by awayteam\n\nplt.figure(figsize=(20, 10))\nsns.barplot(x=df['FTAG'].groupby(df['AwayTeam']).mean().sort_values(ascending=False).index,y=df['FTAG'].groupby(df['AwayTeam']).mean().sort_values(ascending=False))\nplt.title('Mean of Goals Scored by Away Team')\nplt.xlabel('Away Team')\nplt.ylabel('Goals')\nplt.xticks(rotation=90)\nplt.show()"

In [173]:
'''# Mean of goals scored by hometeam

plt.figure(figsize=(20, 10))
sns.barplot(x=df['FTHG'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['FTHG'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))
plt.title('Mean of Goals Scored by Home Team')
plt.xlabel('Home Team')
plt.ylabel('Goals')
plt.xticks(rotation=90)
plt.show()
'''


"# Mean of goals scored by hometeam\n\nplt.figure(figsize=(20, 10))\nsns.barplot(x=df['FTHG'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['FTHG'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))\nplt.title('Mean of Goals Scored by Home Team')\nplt.xlabel('Home Team')\nplt.ylabel('Goals')\nplt.xticks(rotation=90)\nplt.show()\n"

In [174]:
'''# Mean of shots on target by awayteam

plt.figure(figsize=(20, 10))
sns.barplot(x=df['AST'].groupby(df['AwayTeam']).mean().sort_values(ascending=False).index,y=df['AST'].groupby(df['AwayTeam']).mean().sort_values(ascending=False))
plt.title('Mean of Shots on Target by Away Team')
plt.xlabel('Away Team')
plt.ylabel('Shots on Target')
plt.xticks(rotation=90)
plt.show()'''


"# Mean of shots on target by awayteam\n\nplt.figure(figsize=(20, 10))\nsns.barplot(x=df['AST'].groupby(df['AwayTeam']).mean().sort_values(ascending=False).index,y=df['AST'].groupby(df['AwayTeam']).mean().sort_values(ascending=False))\nplt.title('Mean of Shots on Target by Away Team')\nplt.xlabel('Away Team')\nplt.ylabel('Shots on Target')\nplt.xticks(rotation=90)\nplt.show()"

In [175]:
'''# Mean of shots on target by hometeam

plt.figure(figsize=(20, 10))
sns.barplot(x=df['HST'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['HST'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))
plt.title('Mean of Shots on Target by Home Team')
plt.xlabel('Home Team')
plt.ylabel('Shots on Target')
plt.xticks(rotation=90)
plt.show()'''


"# Mean of shots on target by hometeam\n\nplt.figure(figsize=(20, 10))\nsns.barplot(x=df['HST'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['HST'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))\nplt.title('Mean of Shots on Target by Home Team')\nplt.xlabel('Home Team')\nplt.ylabel('Shots on Target')\nplt.xticks(rotation=90)\nplt.show()"

In [176]:
'''# Head-to-Head Records: Historical performance between the two teams.

# HomeTeam vs AwayTeam

plt.figure(figsize=(20, 10))
heatmap_data = pd.crosstab(df['HomeTeam'], df['AwayTeam'])
sns.heatmap(heatmap_data, annot=True, cmap='Blues', fmt="d")
plt.title('Heatmap of Home Team vs Away Team')
plt.xlabel('Away Team')
plt.ylabel('Home Team')
plt.xticks(rotation=90)
plt.show()'''


'# Head-to-Head Records: Historical performance between the two teams.\n\n# HomeTeam vs AwayTeam\n\nplt.figure(figsize=(20, 10))\nheatmap_data = pd.crosstab(df[\'HomeTeam\'], df[\'AwayTeam\'])\nsns.heatmap(heatmap_data, annot=True, cmap=\'Blues\', fmt="d")\nplt.title(\'Heatmap of Home Team vs Away Team\')\nplt.xlabel(\'Away Team\')\nplt.ylabel(\'Home Team\')\nplt.xticks(rotation=90)\nplt.show()'

In [177]:
# Streaks for home team and away team

'''plt.figure(figsize=(20, 10))
sns.barplot(x=df['Streak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['Streak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))
plt.title('Streaks for Home Team')
plt.xlabel('Home Team')
plt.ylabel('Streak')
plt.xticks(rotation=90)
plt.show()'''




"plt.figure(figsize=(20, 10))\nsns.barplot(x=df['Streak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['Streak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))\nplt.title('Streaks for Home Team')\nplt.xlabel('Home Team')\nplt.ylabel('Streak')\nplt.xticks(rotation=90)\nplt.show()"

In [178]:
'''# win streaks for home team and away team

plt.figure(figsize=(20, 10))

sns.barplot(x=df['HomeTeamWinStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['HomeTeamWinStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))
plt.title('Win Streaks for Home Team')
plt.xlabel('Home Team')
plt.ylabel('Win Streak')
plt.xticks(rotation=90)
plt.show()'''




"# win streaks for home team and away team\n\nplt.figure(figsize=(20, 10))\n\nsns.barplot(x=df['HomeTeamWinStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['HomeTeamWinStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))\nplt.title('Win Streaks for Home Team')\nplt.xlabel('Home Team')\nplt.ylabel('Win Streak')\nplt.xticks(rotation=90)\nplt.show()"

In [179]:
'''# loss streaks for home team and away team

plt.figure(figsize=(20, 10))
sns.barplot(x=df['HomeTeamLossStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['HomeTeamLossStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))
plt.title('Loss Streaks for Home Team')
plt.xlabel('Home Team')
plt.ylabel('Loss Streak')
plt.xticks(rotation=90)
plt.show()'''



"# loss streaks for home team and away team\n\nplt.figure(figsize=(20, 10))\nsns.barplot(x=df['HomeTeamLossStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False).index,y=df['HomeTeamLossStreak'].groupby(df['HomeTeam']).mean().sort_values(ascending=False))\nplt.title('Loss Streaks for Home Team')\nplt.xlabel('Home Team')\nplt.ylabel('Loss Streak')\nplt.xticks(rotation=90)\nplt.show()"

In [180]:
import pandas as pd

# Create New DataFrame for Model Selection with raw match data and streaks
model_df = pd.DataFrame()

# Add raw features for away teams
model_df['AwayTeam'] = df['AwayTeam']
model_df['AwayGoals'] = df['FTAG']  # Full-time Away Goals
model_df['AwayShotsOnTarget'] = df['AST']  # Away Shots on Target

# Add raw features for home teams
model_df['HomeTeam'] = df['HomeTeam']
model_df['HomeGoals'] = df['FTHG']  # Full-time Home Goals
model_df['HomeShotsOnTarget'] = df['HST']  # Home Shots on Target

# Add full-time result column for raw match outcome
model_df['FullTimeResult'] = df['FTR']  # Full-time result: H (Home win), A (Away win), D (Draw)


# 1. Calculate win and loss streaks for each team

def calculate_streaks(df, team, is_home=True):
    results = df['FTR'][df['HomeTeam'] == team] if is_home else df['FTR'][df['AwayTeam'] == team]
    streaks = []
    streak = 0
    for result in results:
        if (is_home and result == 'H') or (not is_home and result == 'A'):
            streak += 1
        else:
            if streak > 0:
                streaks.append(streak)
            streak = 0
    if streak > 0:
        streaks.append(streak)
    return max(streaks) if streaks else 0

# Pre-calculate win and loss streaks for each team
home_team_win_streaks = {team: calculate_streaks(df, team, is_home=True) for team in df['HomeTeam'].unique()}
away_team_win_streaks = {team: calculate_streaks(df, team, is_home=False) for team in df['AwayTeam'].unique()}
home_team_loss_streaks = {team: calculate_streaks(df, team, is_home=True) for team in df['HomeTeam'].unique()}
away_team_loss_streaks = {team: calculate_streaks(df, team, is_home=False) for team in df['AwayTeam'].unique()}

# Map streaks to the DataFrame
model_df['HomeTeamWinStreak'] = df['HomeTeam'].map(home_team_win_streaks)
model_df['AwayTeamWinStreak'] = df['AwayTeam'].map(away_team_win_streaks)
model_df['HomeTeamLossStreak'] = df['HomeTeam'].map(home_team_loss_streaks)
model_df['AwayTeamLossStreak'] = df['AwayTeam'].map(away_team_loss_streaks)

# Display the first few rows of the new DataFrame
model_df.head()



,AwayTeam,AwayGoals,AwayShotsOnTarget,HomeTeam,HomeGoals,HomeShotsOnTarget,FullTimeResult,HomeTeamWinStreak,AwayTeamWinStreak,HomeTeamLossStreak,AwayTeamLossStreak
0,Anderlecht,1.0,NaN,Mechelen,0.0,NaN,A,6,12,6,12
1,Roeselare,1.0,NaN,Charleroi,1.0,NaN,D,9,1,9,1
2,Bergen,1.0,NaN,Club Brugge,2.0,NaN,H,21,3,21,3
3,Germinal,2.0,NaN,Dender,1.0,NaN,A,2,3,2,3
4,Westerlo,1.0,NaN,FC Brussels,0.0,NaN,A,3,3,3,3


In [160]:
model_df.to_csv('../Preprocessing/model_df.csv', index=False)